# Imports

In [ ]:
import sys
import copy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from model.base import CovidModel, SimulationParameters, set_parameters, normal_ci, logger
from utils import BasicStatistics, RemovePolicy, Propaganda, setup_city_layout, confidence_interval
from model.utils import SocialPolicy
from model.debugutils import DebugUtils

# Simulation setup

In [ ]:
population_size = 10000
simulation_cycles = 90 # days
multiple_runs = 10

# Scenario 1 - Parameters settings

Default parameters. Unless explicitly overriden, these values are used in all scenarios

In [ ]:
basic_parameters = SimulationParameters(
    social_policies = [
       SocialPolicy.LOCKDOWN_OFFICE,
       SocialPolicy.LOCKDOWN_FACTORY,
       SocialPolicy.LOCKDOWN_RETAIL,
       SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL,
       SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL,
       SocialPolicy.LOCKDOWN_HIGH_SCHOOL,
       SocialPolicy.SOCIAL_DISTANCING
    ],
    hospitalization_capacity = 0.1,
    icu_capacity = 0.01,
    mask_user_rate = 0.5,
    mask_efficacy = 0.4,
    imune_rate = 0.01,
    initial_infection_rate = 0.01,
    latency_period_shape = 3,
    latency_period_scale = 1, 
    incubation_period_shape = 6,
    incubation_period_scale = 1, 
    mild_period_duration_shape = 14,
    mild_period_duration_scale = 1,
    hospitalization_period_duration_shape = 12,
    hospitalization_period_duration_scale = 1,
    symptomatic_isolation_rate = 0.1,
    asymptomatic_contagion_probability = 0.1,
    risk_tolerance_mean = 0.5,
    risk_tolerance_stdev = 0.1,
    herding_behavior_mean = 0.5,
    herding_behavior_stdev = 0.1,
    allowed_restaurant_capacity = 1.0, # valid values: {1.0, 0.50, 0.25}
)

## Scenario 1 - Setup of monitored variables

In [ ]:
sc = 1

basic_r0 = []
count_by_age_group = [[], [], [], [], [], [], [], [], [], []]
deaths_by_age_group = [[], [], [], [], [], [], [], [], [], []]
hospitalized_by_age_group = [[], [], [], [], [], [], [], [], [], []]
icu_by_age_group = [[], [], [], [], [], [], [], [], [], []]
deaths_total = []
hospitalized_total = []
icu_total = []
infected_per_day = {}
infected_per_day[sc] = []
for i in range(simulation_cycles):
    infected_per_day[sc].append([])
symptomatic_per_day = {}
symptomatic_per_day[sc] = []
for i in range(simulation_cycles):
    symptomatic_per_day[sc].append([])

# Scenario 1 - Run simulation multiple times

In [ ]:
%%time
for k in range(multiple_runs):
    print(f"{k + 1}/{multiple_runs}")
    params = copy.deepcopy(basic_parameters)
    set_parameters(params)
    model = CovidModel()
    setup_city_layout(model, population_size)
    model.add_listener(RemovePolicy(model, SocialPolicy.LOCKDOWN_ELEMENTARY_SCHOOL, 30))
    model.add_listener(RemovePolicy(model, SocialPolicy.LOCKDOWN_MIDDLE_SCHOOL, 60))
    model.add_listener(RemovePolicy(model, SocialPolicy.LOCKDOWN_HIGH_SCHOOL, 90))
    statistics = BasicStatistics(model)
    model.add_listener(statistics)
    debug = DebugUtils(model)
    model.add_listener(debug)
    logger().model = model
    for i in range(simulation_cycles):
        model.step()
    debug.update_infection_status()
    basic_r0.append(debug.get_R0_stats()['infections'].mean())
    stats = debug.get_age_group_stats()
    infected = stats['infected'].sum() + stats['recovered'].sum()
    deaths_total.append(stats['deaths'].sum() / infected)
    hospitalized_total.append(stats['hospitalized'].sum() / infected)
    icu_total.append(stats['icu'].sum() / infected)
    for i in range(10):
        infected = stats['infected'][i] + stats['recovered'][i]
        deaths_by_age_group[i].append(0 if infected == 0 else stats['deaths'][i] / infected)
        hospitalized_by_age_group[i].append(0 if infected == 0 else stats['hospitalized'][i] / infected)
        icu_by_age_group[i].append(0 if infected == 0 else stats['icu'][i] / infected)
        count_by_age_group[i].append(stats['count'][i])
    for i in range(simulation_cycles):
        infected_per_day[sc][i].append(debug.new_infections[i + 1])
    new_symptomatics = debug.get_new_symptomatic_stats(simulation_cycles)
    for i in range(simulation_cycles):
        symptomatic_per_day[sc][i].append(new_symptomatics[i])
    statistics.export_chart(f'run_{sc}_{k}.png')
    statistics.export_csv(f'run_{sc}_{k}.csv')

# Scenario 1 - Print results

In [ ]:
print(confidence_interval(basic_r0))

In [ ]:
for i in range(10):
    print(f"{i * 10} to {i * 10 + 9}: {confidence_interval(count_by_age_group[i])}")

In [ ]:
print(f"Total: {confidence_interval(deaths_total)}")
for i in range(10):
    print(f"{i * 10} to {i * 10 + 9}: {confidence_interval(deaths_by_age_group[i])}")

In [ ]:
print(f"Total: {confidence_interval(hospitalized_total)}")
for i in range(10):
    print(f"{i * 10} to {i * 10 + 9}: {confidence_interval(hospitalized_by_age_group[i])}")

In [ ]:
print(f"Total: {confidence_interval(icu_total)}")
for i in range(10):
    print(f"{i * 10} to {i * 10 + 9}: {confidence_interval(icu_by_age_group[i])}")

# Plot results

In [ ]:
mean = np.mean(infected_per_day[1], axis=1)
stdev = np.std(infected_per_day[1], axis=1)
window = 7
ma_mean = np.convolve(mean, np.ones((window,))/window, mode='valid')
ma_stdev = np.convolve(stdev, np.ones((window,))/window, mode='valid')

plt.plot(ma_mean)

#plt.fill_between(range(3), mean-standard_dev, mean+standard_dev, alpha = 0.5)
plt.show()

In [ ]:
mean = np.mean(symptomatic_per_day[1], axis=1)
stdev = np.std(symptomatic_per_day[1], axis=1)
window = 7
ma_mean = np.convolve(mean, np.ones((window,))/window, mode='valid')
ma_stdev = np.convolve(stdev, np.ones((window,))/window, mode='valid')

plt.plot(ma_mean)

#plt.fill_between(range(3), mean-standard_dev, mean+standard_dev, alpha = 0.5)
plt.show()

In [ ]:
print(infected_per_day[1])